In [1]:
from indra_cogex.client import Neo4jClient
import indra_cogex.info
from indra_cogex.client.curation import PHOSPHATASE_CURIES
from rich import print
from collections import Counter, defaultdict
import pandas as pd
from indra.databases import hgnc_client
from indra_cogex.apps.curator.curation_cache import CurationCache

indra_cogex.info.env()

Key,Value
OS,posix
Platform,Darwin
Release,21.4.0
Time,Tue Apr 19 16:46:19 2022
Python,3.9.12
INDRA Version,1.21.0
INDRA Git Hash,fbdf9f50d128b5f22a38f3293ff8a2e3e36715a5
INDRA CoGEx Version,1.0.0
INDRA CoGEx Git Hash,ce19bcb92d8a34e85b9ea470c3b9c098fa49d09d
INDRA CoGEx Branch,main


In [5]:
all_hgnc_ids = set(hgnc_client.hgnc_ids.values())

In [2]:
client = Neo4jClient()

INFO: [2022-04-19 16:46:19] indra_cogex.client.neo4j_client - Using configured URL for INDRA neo4j connection
INFO: [2022-04-19 16:46:19] indra_cogex.client.neo4j_client - Using configured credentials for INDRA neo4j connection


In [8]:
%%time
query = f"""\
MATCH (a:BioEntity)-[r:indra_rel]->(b:BioEntity)
WHERE
    a.id STARTS WITH "hgnc"
    AND a.id <> b.id
RETURN a.id, a.name, count(r)
"""

res = list(client.query_tx(query))

CPU times: user 707 ms, sys: 65 ms, total: 772 ms
Wall time: 13 s


In [9]:
known_genes = {row[0].removeprefix("hgnc:") for row in res}

dark_genes = all_hgnc_ids - known_genes
print(f"There are {len(dark_genes):,} dark genes of {len(all_hgnc_ids):,} genes")

There are 24,954 dark genes of 44,903 genes

In [13]:
known_gene_type_counter = Counter(
    hgnc_client.get_gene_type(known_gene) for known_gene in known_genes
)
print(known_gene_type_counter)

Counter({
    'gene with protein product': 17514,
    'unknown': 64,
    'RNA, long non-coding': 844,
    'phenotype only': 100,
    'RNA, micro': 755,
    'readthrough': 23,
    'immunoglobulin gene': 42,
    'pseudogene': 404,
    'T cell receptor gene': 15,
    'protocadherin': 28,
    'RNA, small nuclear': 15,
    'RNA, small nucleolar': 74,
    'RNA, transfer': 12,
    'RNA, cluster': 4,
    'endogenous retrovirus': 15,
    'RNA, Y': 4,
    'virus integration site': 3,
    'region': 8,
    'complex locus constituent': 24,
    'RNA, ribosomal': 5,
    'fragile site': 2,
    'RNA, misc': 4,
    'RNA, vault': 3,
    'T cell receptor pseudogene': 1,
    'transposable element': 1,
    'immunoglobulin pseudogene': 1
})

In [12]:
dark_gene_type_counter = Counter(
    hgnc_client.get_gene_type(dark_gene) for dark_gene in dark_genes
)
print(dark_gene_type_counter)

Counter({
    'pseudogene': 13688,
    'RNA, micro': 1215,
    'gene with protein product': 1833,
    'RNA, long non-coding': 4798,
    'phenotype only': 472,
    'unknown': 551,
    'immunoglobulin pseudogene': 202,
    'RNA, small nucleolar': 495,
    'RNA, transfer': 603,
    'immunoglobulin gene': 188,
    'RNA, cluster': 123,
    'virus integration site': 5,
    'region': 74,
    'endogenous retrovirus': 102,
    'T cell receptor gene': 191,
    'fragile site': 116,
    'RNA, misc': 26,
    'protocadherin': 11,
    'readthrough': 117,
    'RNA, small nuclear': 43,
    'T cell receptor pseudogene': 36,
    'transposable element': 3,
    'RNA, ribosomal': 55,
    'complex locus constituent': 6,
    'RNA, vault': 1
})